# Telemetry Pipeline - Model Fine Tuning Workflow
This notebook generates synthetic switch telemetry data, prepares a model for GPT-2 fine-tuning 

In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def generate_synthetic_switch_telemetry(num_samples=100):
    np.random.seed(42)
    
    start_time = datetime.now()
    timestamps = [start_time - timedelta(minutes=5 * i) for i in range(num_samples)]
    names = [f"QSFP-{i%32+1}" for i in range(num_samples)]

    data = {
        'timestamp': timestamps,
        'name': names,
        'temp': np.random.uniform(20, 80, num_samples),
        'trans-volt': np.random.uniform(3.2, 3.5, num_samples)
    }

    for ch in range(1, 5):
        data[f'channel_{ch}_in_pwr'] = np.random.uniform(-3, 3, num_samples)
        data[f'channel_{ch}_out_pwr'] = np.random.uniform(-3, 3, num_samples)
        data[f'channel_{ch}_laser_bias_cur'] = np.random.uniform(5, 10, num_samples)

    df = pd.DataFrame(data)
    df['timestamp'] = df['timestamp'].astype(str)
    df.to_csv('synthetic_switch_telemetry.csv', index=False)
    return df

df = generate_synthetic_switch_telemetry(200)
df.head()
    

In [ ]:

import json

def classify_anomaly(row):
    anomalies = []
    if row['temp'] > 75:
        anomalies.append("Overheating")
    if row['trans-volt'] < 3.1 or row['trans-volt'] > 3.5:
        anomalies.append("Voltage Drift")
    for ch in range(1, 5):
        if row[f'channel_{ch}_laser_bias_cur'] > 9:
            anomalies.append(f"Channel {ch} Bias Current Spike")
        if row[f'channel_{ch}_out_pwr'] < -3:
            anomalies.append(f"Channel {ch} Power Loss")
    return "Anomalous - " + ', '.join(anomalies) if anomalies else "Normal"

def row_to_prompt(row):
    prompt = (
        f"Telemetry Report:\n"
        f"- Timestamp: {row['timestamp']}\n"
        f"- Module: {row['name']}\n"
        f"- Temperature: {row['temp']:.2f}°C\n"
        f"- Transceiver Voltage: {row['trans-volt']:.2f}V\n"
    )
    for ch in range(1, 5):
        prompt += (
            f"- Channel {ch} Input Power: {row[f'channel_{ch}_in_pwr']:.2f} dBm\n"
            f"- Channel {ch} Output Power: {row[f'channel_{ch}_out_pwr']:.2f} dBm\n"
            f"- Channel {ch} Laser Bias Current: {row[f'channel_{ch}_laser_bias_cur']:.2f} mA\n"
        )
    prompt += "\nIs this normal or anomalous?"
    return prompt

with open('train.jsonl', 'w') as f:
    for _, row in df.iterrows():
        json.dump({"prompt": row_to_prompt(row), "response": classify_anomaly(row)}, f)
        f.write('\n')

print("✅ train.jsonl created.")
    

# Safe Fine-Tuning and Saving 

In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset

# --- Debug GPU Information ---
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"PyTorch Version: {torch.__version__}")
    os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # Force clearer error reporting from CUDA

# --- Load Dataset ---
dataset = load_dataset('json', data_files={'train': 'train.jsonl'})
train_test_split = dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# --- Load Tokenizer ---
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure tokenizer has padding token (GPT-2 does not have one by default)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- Load Model Safely ---
# Set dtype explicitly to match intended precision (can be float16 if using fp16 training)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)

# Move to GPU after verifying load works
model = model.to("cuda")

# --- Tokenization Helper ---
def concatenate_prompt_response(examples):
    combined = [
        f"prompt: {p}\nresponse: {r}" for p, r in zip(examples['prompt'], examples['response'])
    ]
    return tokenizer(combined, truncation=True, max_length=512)

# Tokenize datasets
tokenized_train = train_dataset.map(concatenate_prompt_response, batched=True)
tokenized_eval = eval_dataset.map(concatenate_prompt_response, batched=True)

# --- Data Collator (dynamic padding) ---
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# --- Training Arguments ---
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,  # Use mixed precision
    logging_dir="./logs",
    logging_steps=10,
    report_to="tensorboard"
)

# --- Trainer Setup ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# --- Train Model ---
trainer.train()

# --- Safe Save (CPU-based) ---
print("✅ Training complete. Saving model to CPU...")

model = model.to("cpu")
model.save_pretrained("fine_tuned_gpt2_telemetry")
tokenizer.save_pretrained("fine_tuned_gpt2_telemetry")

print("✅ Model and tokenizer saved safely to 'fine_tuned_gpt2_telemetry'.")

# --- Post-save Reload Test ---
print("✅ Reloading saved model for sanity check...")

reloaded_model = AutoModelForCausalLM.from_pretrained("fine_tuned_gpt2_telemetry", torch_dtype=torch.float32)
reloaded_model = reloaded_model.to("cuda")  # Move back to GPU

reloaded_tokenizer = AutoTokenizer.from_pretrained("fine_tuned_gpt2_telemetry")

# Quick inference test to confirm save/load worked
test_input = "prompt: What is knowledge distillation?\nresponse:"
inputs = reloaded_tokenizer(test_input, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = reloaded_model(**inputs)

print(f"✅ Reloaded model test passed. Output shape: {outputs.logits.shape}")


# Test for Model Corruption

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"PyTorch Version: {torch.__version__}")

try:
    teacher = AutoModelForCausalLM.from_pretrained("./fine_tuned_gpt2_telemetry", device_map=None)
    print("✅ Model loaded successfully to CPU.")

    teacher = teacher.to("cuda")
    print("✅ Model moved to GPU successfully.")
except Exception as e:
    print(f"❌ Error during model load/move: {e}")


CUDA Available: True
CUDA Device: Quadro M1000M
CUDA Version: 11.7
PyTorch Version: 2.0.0+cu117


C:\Miniconda3\envs\sentence-transformers\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Miniconda3\envs\sentence-transformers\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedba

✅ Model loaded successfully to CPU.
✅ Model moved to GPU successfully.
